# Sector Distribution

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y='GICS Sector', data=securities_df, order=securities_df['GICS Sector'].value_counts().index)
plt.title('Number of Companies by Sector')
plt.xlabel('Number of Companies')
plt.ylabel('Sector')
plt.show()


# Merge Datasets

In [ ]:
# Merge prices and prices_split_adjusted on 'symbol' and 'date'
merged_prices = pd.merge(prices_df, prices_split_adjusted_df, on=['symbol', 'date'], suffixes=('', '_adjusted'))

# Merge with securities on 'symbol' and 'Ticker symbol'
merged_data = pd.merge(merged_prices, securities_df, left_on='symbol', right_on='Ticker symbol')

# Merge with fundamentals on 'symbol' and 'Ticker symbol'
merged_data = pd.merge(merged_prices, fundamentals_df, left_on='symbol', right_on='Ticker Symbol')


#Distribution of Adjusted and Unadjusted Closing Prices

In [ ]:
# Set plot style
sns.set(style="whitegrid")

# Distribution of Adjusted and Unadjusted Closing Prices
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.histplot(merged_data['close'], bins=50, kde=True)
plt.title('Distribution of Unadjusted Closing Prices')

plt.subplot(1, 2, 2)
sns.histplot(merged_data['close_adjusted'], bins=50, kde=True)
plt.title('Distribution of Adjusted Closing Prices')
plt.show()


# Trends Over Time for Adjusted vs. Unadjusted Prices

In [ ]:
# Select a few symbols for demonstration
symbols_to_plot = ['AAPL', 'MSFT', 'GOOGL']  # Example symbols
plt.figure(figsize=(14, 8))

for symbol in symbols_to_plot:
    subset = merged_data[merged_data['symbol'] == symbol]
    plt.plot(subset['date'], subset['close'], label=f'{symbol} - Unadjusted')
    plt.plot(subset['date'], subset['close_adjusted'], label=f'{symbol} - Adjusted', linestyle='--')

plt.title('Historical Price Trends (Adjusted vs. Unadjusted)')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


# Correlation Matrix

In [ ]:
# Exclude 'symbol' and 'date' columns from correlation calculation
cols_to_exclude = ['symbol', 'date', 'Ticker Symbol', "For Year", "Period Ending", "Unnamed: 0"]
cols_to_include = [col for col in merged_data.columns if col not in cols_to_exclude]
corr_matrix = merged_data[cols_to_include].corr()


In [ ]:
# Here's a subset of relevant columns based on the correlation matrix you've shown
relevant_columns = ['open','open_adjusted','close', 'close_adjusted', 'high','high_adjusted', 'low','low_adjusted', 'volume', "volume_adjusted", 'Earnings Per Share']

# Subset the correlation matrix
relevant_corr_matrix = corr_matrix.loc[relevant_columns, relevant_columns]

# Plotting the heatmap for relevant correlations
plt.figure(figsize=(10, 8))
sns.heatmap(relevant_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap of Key Variables for Stock Price Prediction')
plt.show()


## Pre-processing

## Feature Selection

In [ ]:
# Selecting initial important features
initial_features = [
    'date', 'symbol', 'open', 'close','low', 'high', 'volume',
    'Earnings Before Interest and Tax', 'Net Income', 'Operating Income', 'Gross Profit',
    'Profit Margin', 'Gross Margin', 'Operating Margin', 'Pre-Tax Margin', 'After Tax ROE',
    'Earnings Per Share', 'Current Ratio', 'Quick Ratio', 'Cash Ratio', 'Long-Term Debt',
    'Short-Term Debt / Current Portion of Long-Term Debt', 'Total Liabilities', 'Total Equity',
    'Net Cash Flow', 'Net Cash Flow-Operating', 'Net Cash Flows-Financing',
    'Net Cash Flows-Investing', 'Estimated Shares Outstanding', 'Sale and Purchase of Stock'
]

# Drop rows with missing values in the selected features
df_sf = merged_data[initial_features].dropna()

# Convert 'date' column to datetime if not already done
df_sf['date'] = pd.to_datetime(df_sf['date'])

# Check the updated dataframe
df_sf

In [ ]:
# Resample to monthly data for each symbol
df_monthly = df_sf.groupby(['symbol', pd.Grouper(key='date', freq='M')]).agg({
    'open': 'first',
    'close': 'last',
    'low': 'min',
    'high': 'max',
    'volume': 'sum',
    'Earnings Before Interest and Tax': 'mean',
    'Net Income': 'mean',
    'Operating Income': 'mean',
    'Gross Profit': 'mean',
    'Profit Margin': 'mean',
    'Gross Margin': 'mean',
    'Operating Margin': 'mean',
    'Pre-Tax Margin': 'mean',
    'After Tax ROE': 'mean',
    'Earnings Per Share': 'mean',
    'Current Ratio': 'mean',
    'Quick Ratio': 'mean',
    'Cash Ratio': 'mean',
    'Long-Term Debt': 'mean',
    'Short-Term Debt / Current Portion of Long-Term Debt': 'mean',
    'Total Liabilities': 'mean',
    'Total Equity': 'mean',
    'Net Cash Flow': 'mean',
    'Net Cash Flow-Operating': 'mean',
    'Net Cash Flows-Financing': 'mean',
    'Net Cash Flows-Investing': 'mean',
    'Estimated Shares Outstanding': 'mean',
    'Sale and Purchase of Stock': 'mean'
}).reset_index()

df_monthly
